In [488]:
import bs4 as bs
import urllib.request
import collections
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
import csv

def crawl(url):
    
    src=urllib.request.urlopen(url).read()
    soup=bs.BeautifulSoup(src,'lxml')
    process(soup)

def process(soup):
    
    stop_words=set(stopwords.words('english'))
    ps=PorterStemmer()
    lemmatizer=WordNetLemmatizer()

    file=open("text.txt","w")
    

    a=[]
    title=soup.find('title').text
    t=re.sub(r'^\w]', ' ', title)
##    file.write(t)
##    print(t)
    t_t=word_tokenize(t)
##    print(t_t)
    t1=[i for i in t_t if i.lower() not in stop_words]
    cruft = '&`©@%?-,_./()[]{}\^\\n+*^~:;!""``.' + "''...,'s<>'â{}=n't"
    t2=[i for i in t1 if i not in cruft]
    a.append(t)
    txt=soup.find('p')
##    print(txt.text)
    for p in soup.findAll('p'):
        p=p.text
        t=re.sub(r'^\w]', ' ', p)
##        file.append(t)
##        print(t)
##        p_t=word_tokenize(t)
##        t3=[i for i in p_t if i.lower()  not in stop_words]
##        t4=[i for i in t3 if i not in cruft]
        a.append(t)
##    file.close()
##    print(a)
    for i in a:
        print(i)
        file.write(i)
    file.close()
      



def main():
    url='http://www.nust.edu.pk/INSTITUTIONS/Colleges/CEME/Pages/default.aspx'
    crawl(url)


main()



	
	College of Electrical & Mechanical Engineering (CEME) - National University of Sciences and Technology (NUST)




<h2> Load review data through Sklearn </h2>

In [489]:
import sklearn
from sklearn.datasets import load_files

In [490]:
reviewdir = r'reviews'

In [491]:
# loading all files as training data. 
review_train = load_files(reviewdir, shuffle=True)

In [492]:
len(review_train.data)

134

In [493]:
# target names ("classes") are automatically generated from subfolder names
review_train.target_names

['Not Suspicious', 'suspisious']

In [494]:
# First file seems to be about a reviews. 
review_train.data[0][:500]

b'Tuesday\xe2\x80\x99s deadly coordinated bomb attacks against soft targets in Iraq bear the mark of a simmering Sunni insurgency. And despite the successes of the 2007 troop surge, our unwillingness to tackle the root of the sectarian divisions in Iraq will mean further bloodshed for the people of that country.\n\nAs I argue in my book The Sunni-Shia Conflict: Understanding Sectarian Violence in the Middle East, one of the Bush administration\xe2\x80\x99s most tragic blunders (aside from the invasion proper) was its'

In [495]:
# first file is in "neg" folder
review_train.filenames[0]

'reviews/suspisious/cv037_18510.txt'

In [496]:
# first file is a negative review and is mapped to 0 index 'neg' in target_names
review_train.target[0]

1

<h2> Counter Vectorizer </h2>

In [497]:
# import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [498]:
# Turn off pretty printing of jupyter notebook... it generates long lines
%pprint

Pretty printing has been turned OFF


In [499]:
import nltk

In [500]:
sents = ['Shia sunni brotherhood.',
         'Muslim unity and peace.',
        "This is what Islam teaches."]

In [501]:
# Initialize a CoutVectorizer to use NLTK's tokenizer instead of its 
# default one (which ignores punctuation and stopwords). 
# Minimum document frequency set to 1. 
foovec = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize)

In [502]:
# sents turned into sparse vector of word frequency counts
sents_counts = foovec.fit_transform(sents)
# foovec now contains vocab dictionary which maps unique words to indexes
foovec.vocabulary_

{'shia': 7, 'sunni': 8, 'brotherhood': 2, '.': 0, 'muslim': 5, 'unity': 11, 'and': 1, 'peace': 6, 'this': 10, 'is': 3, 'what': 12, 'islam': 4, 'teaches': 9}

In [503]:
# sents_counts has a dimension of 3 (document count) by 19 (# of unique words)
sents_counts.shape

(3, 13)

In [504]:
# this vector is small enough to view in full! 
sents_counts.toarray()

array([[1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [505]:
# Convert raw frequency counts into TF-IDF (Term Frequency -- Inverse Document Frequency) values
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
sents_tfidf = tfidf_transformer.fit_transform(sents_counts)

In [506]:
# TF-IDF values
# raw counts have been normalized against document length, 
# terms that are found across many docs are weighted down
sents_tfidf.toarray()

array([[0.32274454, 0.        , 0.54645401, 0.        , 0.        ,
        0.        , 0.        , 0.54645401, 0.54645401, 0.        ,
        0.        , 0.        , 0.        ],
       [0.28321692, 0.47952794, 0.        , 0.        , 0.        ,
        0.47952794, 0.47952794, 0.        , 0.        , 0.        ,
        0.        , 0.47952794, 0.        ],
       [0.2553736 , 0.        , 0.        , 0.43238509, 0.43238509,
        0.        , 0.        , 0.        , 0.        , 0.43238509,
        0.43238509, 0.        , 0.43238509]])

<h2> Transforming  Reviews </h2>


In [507]:
# initialize movie_vector object, and then turn movie train data into a vector 
review_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all 25K words. 82.2% acc.
# movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features = 3000) # use top 3000 words only. 78.5% acc.
review_counts = review_vec.fit_transform(review_train.data)

In [508]:
# 'screen' is found in the corpus, mapped to index 19637
review_vec.vocabulary_.get('screen')

In [509]:
# Likewise, Mr. Steven Seagal is present...
review_vec.vocabulary_.get('seagal')

In [510]:
# huge dimensions! 2,000 documents, 25K unique terms. 
review_counts.shape

(134, 5221)

In [511]:
# Convert raw frequency counts into TF-IDF values
tfidf_transformer = TfidfTransformer()
review_tfidf = tfidf_transformer.fit_transform(review_counts)

In [512]:
# Same dimensions, now with tf-idf values instead of raw frequency counts
review_tfidf.shape

(134, 5221)

<h2> Training and Testing Naive Bayes Classifier </h2>

In [513]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

In [514]:
# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(
    review_tfidf, review_train.target, test_size = 0.20, random_state = 12)

In [515]:
# Train a Multimoda Naive Bayes classifier
clf = MultinomialNB().fit(docs_train, y_train)

In [516]:
# Predicting the Test set results, find accuracy
y_pred = clf.predict(docs_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8148148148148148

In [517]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 7,  5],
       [ 0, 15]])

<h2> Trying Classifier on Reviews </h2>

In [518]:
# very short 
with open("/home/naqqash/Desktop/Project ki khap/Link-Python Project/text.txt", "r") as file:
    data=file.read().replace('\n', '')

reviews_new = [data]
reviews_new_counts = review_vec.transform(reviews_new)
reviews_new_tfidf = tfidf_transformer.transform(reviews_new_counts)

In [519]:
# have classifier make a prediction
pred = clf.predict(reviews_new_tfidf)

In [520]:
# print out results
for review, category in zip(reviews_new, pred):
    print('The Website you are trying to access contains %s data' % (review_train.target_names[category]))

The Website you are trying to access contains Not Suspicious data
